In [3]:
import os
import pandas as pd

from PIL import Image
import io
from tqdm import tqdm
from torchvision import transforms
import numpy as np
import os
from datasets import Dataset, concatenate_datasets, Features, Image, Value

In [4]:
# Load the dataframe
df = pd.read_csv("/mnt/data1/sabbih/datasets/dash-storyboards/master_data_new.tsv", sep='\t', header=None)

# Rename columns as per the script
df.rename(columns={0: "fname", 1: "story_board", 2: "final_image", 3: "caption"}, inplace=True)

# Convert all ints in fname column to 00000 format
df["fname"] = df["fname"].apply(lambda x: str(x).zfill(5))

# Define the path to the storyboard and images directories
fpath = "/mnt/data1/sabbih/datasets/dash-storyboards"
storyboard_dir = os.path.join(fpath, 'storyboard')
final_image_dir = os.path.join(fpath, 'final_image')

story_boards = os.listdir(storyboard_dir)
images = os.listdir(final_image_dir)
df.head()

,fname,story_board,final_image,caption
0,00000,https://content.dashtoon.ai/stability-images/6...,https://content.dashtoon.ai/user-uploaded-imag...,"Kelly (SB2) Adult, Female, blue sleeveless min..."
1,00001,https://content.dashtoon.ai/stability-images/0...,https://content.dashtoon.ai/user-uploaded-imag...,"Emily Smith Caucasian, Young Adult, Female, Bl..."
2,00002,https://content.dashtoon.ai/stability-images/2...,https://content.dashtoon.ai/user-uploaded-imag...,"Dark blue sparkly stage, white curtains"
3,00003,https://content.dashtoon.ai/stability-images/3...,https://content.dashtoon.ai/user-uploaded-imag...,"Kelly (SB2) Adult, Female, sleeveless blue gow..."
4,00004,https://content.dashtoon.ai/stability-images/a...,https://content.dashtoon.ai/user-uploaded-imag...,"Chinmay Misc Old Female, Eyes closed, Gray Hai..."


In [5]:
# Convert the lists of storyboards and images to sets for faster lookup
story_boards_set = set(story_boards)
images_set = set(images)

# # Filter the dataframe to include only rows where the file name exists in both sets
filtered_df = df[df["fname"].apply(lambda x: f"{x}.webp" in story_boards_set and f"{x}.webp" in images_set)]

# # Update the 'story_board' and 'final_image' columns in the filtered dataframe
filtered_df['story_board'] = filtered_df['fname'].apply(lambda x: f"storyboard/{x}.webp")
filtered_df['final_image'] = filtered_df['fname'].apply(lambda x: f"final_image/{x}.webp")

filtered_df.head()  # Display the first few rows of the updated dataframe

/tmp/ipykernel_1254580/3881298935.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['story_board'] = filtered_df['fname'].apply(lambda x: f"storyboard/{x}.webp")
/tmp/ipykernel_1254580/3881298935.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['final_image'] = filtered_df['fname'].apply(lambda x: f"final_image/{x}.webp")


,fname,story_board,final_image,caption
0,00000,storyboard/00000.webp,final_image/00000.webp,"Kelly (SB2) Adult, Female, blue sleeveless min..."
1,00001,storyboard/00001.webp,final_image/00001.webp,"Emily Smith Caucasian, Young Adult, Female, Bl..."
2,00002,storyboard/00002.webp,final_image/00002.webp,"Dark blue sparkly stage, white curtains"
3,00003,storyboard/00003.webp,final_image/00003.webp,"Kelly (SB2) Adult, Female, sleeveless blue gow..."
4,00004,storyboard/00004.webp,final_image/00004.webp,"Chinmay Misc Old Female, Eyes closed, Gray Hai..."


In [8]:
# Load your existing DataFrame
df = filtered_df.copy()
df.drop(columns=["fname"], inplace=True)
fpath = "/mnt/data1/sabbih/datasets/dash-storyboards"
# Rename columns for clarity
df.rename(columns={"story_board": "storyboard_path", "final_image": "final_image_path"}, inplace=True)
# convert caption column to string
# Update paths to include the full path
df['storyboard_path'] = df['storyboard_path'].apply(lambda x: os.path.join(fpath, x))
df['final_image_path'] = df['final_image_path'].apply(lambda x: os.path.join(fpath, x))

df.dtypes

storyboard_path     object
final_image_path    object
caption             object
dtype: object

In [9]:

df.rename(columns={
    "storyboard_path": "storyboard_image",
    "final_image_path": "final_image",
    "caption": "caption"
}, inplace=True)


# Define the features of the dataset
features = Features({
    'storyboard_image': Image(decode=True, id=None),
    'final_image': Image(decode=True, id=None),
    'caption': Value('string')
})

# Initialize an empty Hugging Face dataset
final_dataset = None

# Define the batch size for iteration
batch_size = 1000  

for start_row in tqdm(range(0, len(df), batch_size)):
    end_row = min(start_row + batch_size, len(df))
    df_batch = df.iloc[start_row:end_row]
    
    # Convert the batch to a Hugging Face dataset with defined features
    batch_dataset = Dataset.from_pandas(df_batch, features=features, preserve_index=False)

    # Append to the final dataset
    if final_dataset is None:
        final_dataset = batch_dataset
    else:
        final_dataset = concatenate_datasets([final_dataset, batch_dataset])

    # Optional: break if you want to test with just one batch
    # break

# Save the final dataset to disk in Arrow format ## uncomment this to actually write the dataset
# final_dataset.save_to_disk("/mnt/data1/sabbih/datasets/dash-storyboards/storyboard_hf_dataset_94487")


100%|██████████| 95/95 [00:00<00:00, 229.90it/s]
